In [1]:
import numpy as np
from scipy.optimize import least_squares
import csv

therm_name = "B57560G11104" # thermistor name
csv_file = open("./" + therm_name + ".csv", "r")
f = csv.DictReader(csv_file, delimiter=",", doublequote=True, lineterminator="\n", quotechar='"', skipinitialspace=True)

number_of_elem = 72 # You must change this value when you use another thermistor property file.

rs = np.zeros(number_of_elem)
tmps = np.zeros(number_of_elem)

for i, row in enumerate(f):
    tmps[i]  = row["T"]
    rs[i]  = row["RT/R25"]

rt = 100000.0 # The resistance of the thermistor at 25°C (unit: ohm).
rs = rs * rt

# model
def steinhart_hart(rs, a, b, c):
    return 1.0 / (a + b * np.log(rs) + c * np.log(rs)**3) - 273.15

def calc_residuals(params, rs, tmps):
    model_tmps = steinhart_hart(rs, params[0], params[1], params[2])
    return model_tmps - tmps

a_init = 7.1 * 10 ** (-4) # initial value of the coefficient A
b_init = 2.2 * 10 ** (-4) # initial value of the coefficient B
c_init = 8.5 * 10 ** (-8) # initial value of the coefficient C

# fitting
res = least_squares(calc_residuals, np.array([a_init, b_init, c_init]), args=(rs, tmps))

a_fit = res.x[0]
b_fit = res.x[1]
c_fit = res.x[2]
tmps_fit = steinhart_hart(rs, a_fit, b_fit, c_fit)
print "a_fit: " + str(a_fit)
print "b_fit: " + str(b_fit)
print "c_fit: " + str(c_fit)

# visualize
import matplotlib.pyplot as plt

plt.plot(rs, tmps)
plt.plot(rs, tmps_fit, linestyle="dashed")
plt.xlabel("resistance[ohm]")
plt.ylabel("temperature[degree]")

# save the fitting result
import json
import collections as cl

coefficients = cl.OrderedDict()
coefficients["A"] = a_fit
coefficients["B"] = b_fit
coefficients["C"] = c_fit

with open("./" + therm_name + ".json","w") as fw:
    json.dump(coefficients, fw, indent=4)


a_fit: 0.0007211385988249983
b_fit: 0.00021700362404422229
c_fit: 8.78016643679097e-08
